In [2]:
import os
import warnings

import pandas as pd

from jre_utils.datapath import factor_data_paths
from jre_utils.config import area_levels, period_cols
from jre_utils.constants import BLACKLISTED_AREAS

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [4]:
population_unprocessed_path = factor_data_paths["unprocessed"]["population"][
    "municipality"
]
migration_unprocessed_path = factor_data_paths["unprocessed"]["migration"][
    "municipality"
]

population_processed_path = factor_data_paths["processed"]["population"]["municipality"]
population_processed_path_all = factor_data_paths["processed"]["population"][
    "submunicipality"
]

migration_processed_path = factor_data_paths["processed"]["migration"]["municipality"]
migration_processed_path_all = factor_data_paths["processed"]["migration"][
    "submunicipality"
]

In [5]:
population_df = pd.read_csv(population_unprocessed_path)
migration_df = pd.read_csv(migration_unprocessed_path)

In [6]:
population_df

,year,area_code,area,population
0,2020,1100,Hokkaido Sapporo-shi,1973395
1,2020,1101,Hokkaido Sapporo-shi Chuo-ku,248680
2,2020,1102,Hokkaido Sapporo-shi Kita-ku,289323
3,2020,1103,Hokkaido Sapporo-shi Higashi-ku,265379
4,2020,1104,Hokkaido Sapporo-shi Shiroishi-ku,211835
...,...,...,...,...
1911,2020,47361,Okinawa-ken Kumejima-cho,7192
1912,2020,47362,Okinawa-ken Yaese-cho,30941
1913,2020,47375,Okinawa-ken Tarama-son,1058
1914,2020,47381,Okinawa-ken Taketomi-cho,3942


In [7]:
population_df = pd.read_csv(population_unprocessed_path)
migration_df = pd.read_csv(migration_unprocessed_path)

population_df = population_df.drop_duplicates(subset=["year", "area_code"])
migration_df = migration_df.drop_duplicates(subset=["year", "area_code"])

population_df = population_df[population_df["year"] == 2020]
combined_df = migration_df.merge(population_df, on=["year", "area_code"], suffixes=["", "_y"], how="left")

combined_df = combined_df[~combined_df["area"].isin(BLACKLISTED_AREAS)]
combined_df = combined_df.sort_values(by=["area_code", "year"], ascending=[False, True])

combined_df["population"] = combined_df.groupby("area_code")["population"].ffill()
combined_df["population"] = combined_df.groupby("area_code")["population"].bfill()

combined_df = combined_df[combined_df["in_migrations"].apply(lambda x: x.isdigit())]
combined_df = combined_df[combined_df["out_migrations"].apply(lambda x: x.isdigit())]

combined_df["in_migrations"] = combined_df["in_migrations"].astype(float)
combined_df["out_migrations"] = combined_df["out_migrations"].astype(float)

combined_df["net_migrations"] = (
    combined_df["in_migrations"] - combined_df["out_migrations"]
)
combined_df["del_population"] = combined_df[
    "net_migrations"
]  # add other factors to improve estimate

combined_df["net_population_chg"] = combined_df["del_population"]

combined_df.loc[combined_df["year"] == 2020, "del_population"] = 0

combined_df_pre = combined_df[combined_df["year"] <= 2020]
combined_df_pre = combined_df_pre.sort_values(
    by=["year", "area_code"], ascending=[False, True]
)
combined_df_pre["cumulative_del_population"] = combined_df_pre.groupby("area_code")[
    "del_population"
].cumsum()
combined_df_pre["population"] = (
    combined_df_pre["population"] - combined_df_pre["cumulative_del_population"]
)

combined_df_post = combined_df[combined_df["year"] > 2020]
combined_df_post = combined_df_post.sort_values(
    by=["year", "area_code"], ascending=[True, True]
)
combined_df_post["cumulative_del_population"] = combined_df_post.groupby("area_code")[
    "del_population"
].cumsum()
combined_df_post["population"] = (
    combined_df_post["population"] + combined_df_post["cumulative_del_population"]
)

combined_df = pd.concat(
    [combined_df_pre, combined_df_post], ignore_index=True
).sort_values(by=["year", "area_code"], ascending=[False, True])

all_area_population_df = combined_df[
    ["year", "area_code", "area", "population"]
]  # save for future use

combined_df = combined_df.drop(columns=["cumulative_del_population"])

In [8]:
combined_df["net_migration_ratio"] = (
    combined_df["net_migrations"] / combined_df["population"]
)
migration_df = combined_df[
    ["year", "area_code", "area", "net_migration_ratio"]
]
population_df = combined_df[["year", "area_code", "area", "population"]]

migration_df.to_csv(migration_processed_path, index=False)
population_df.to_csv(population_processed_path, index=False)
all_area_population_df.to_csv(population_processed_path_all, index=False)

In [9]:
# Different from the one done previously. Understand why
migration_df

,year,area_code,area,net_migration_ratio
47400,2022,1100,Hokkaido Sapporo-shi,0.004506
47401,2022,1101,Hokkaido Sapporo-shi Chuo-ku,0.013748
47402,2022,1102,Hokkaido Sapporo-shi Kita-ku,0.002911
47403,2022,1103,Hokkaido Sapporo-shi Higashi-ku,0.002341
47404,2022,1104,Hokkaido Sapporo-shi Shiroishi-ku,0.002847
...,...,...,...,...
45491,1996,47361,Okinawa-ken Kumejima-cho,-0.007292
45492,1996,47362,Okinawa-ken Yaese-cho,0.012110
45493,1996,47375,Okinawa-ken Tarama-son,-0.031144
45494,1996,47381,Okinawa-ken Taketomi-cho,-0.002947
